In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_excel("C:\\Users\\stadn\\Desktop\\_robot_dane_operator\\wyniki.xlsx")

In [3]:
df = df.drop([0], axis=0)

In [4]:
df["Nr obserwacji"] = np.arange(1, 31)

In [5]:
df.head()

,Nr obserwacji,Operator 1,Unnamed: 2,Operator 2,Unnamed: 4,Operator 3,Unnamed: 6,Operator 4,Unnamed: 8,Operator 5,Unnamed: 10
1,1,167,98,85,121,148,130,168,171,174,117
2,2,138,83,102,128,164,123,208,146,156,73
3,3,101,105,108,133,168,114,184,132,174,115
4,4,120,111,78,116,171,119,207,145,164,115
5,5,135,101,85,110,147,110,201,139,163,120


In [6]:
operator1 = df[["Nr obserwacji", "Operator 1", "Unnamed: 2"]]
operator1.columns = (["Nr obserwacji", "X", "Y"])

In [7]:
operator2 = df[["Nr obserwacji", "Operator 2", "Unnamed: 4"]]
operator2.columns = (["Nr obserwacji", "X", "Y"])

In [8]:
operator3 = df[["Nr obserwacji", "Operator 3", "Unnamed: 6"]]
operator3.columns = (["Nr obserwacji", "X", "Y"])

In [9]:
operator4 = df[["Nr obserwacji", "Operator 4", "Unnamed: 8"]]
operator4.columns = (["Nr obserwacji", "X", "Y"])

In [10]:
operator5 = df[["Nr obserwacji", "Operator 5", "Unnamed: 10"]]
operator5.columns = (["Nr obserwacji", "X", "Y"])

In [11]:
operators = [operator1, operator2, operator3, operator4, operator5]

In [66]:
operator4

,Nr obserwacji,X,Y
1,1,168,171
2,2,208,146
3,3,184,132
4,4,207,145
5,5,201,139
6,6,202,162
7,7,199,150
8,8,209,131
9,9,195,146
10,10,214,134


In [78]:
def predict_y(data, test_size):
    X = data.drop("Y", axis=1)
    y = data["Y"]
    
    poly_converter = PolynomialFeatures(degree=3, include_bias=False)
    poly_converter.fit(X)
    poly_features = poly_converter.transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=test_size, random_state=45)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    pred_y = model.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, pred_y))
    
    
    
    return pred_y

In [79]:
def predict_x(data, test_size):
    X = data.drop("X", axis=1)
    y = data["X"]
    
    poly_converter = PolynomialFeatures(degree=3, include_bias=False)
    poly_converter.fit(X)
    poly_features = poly_converter.transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=test_size, random_state=45)
    
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    pred_x = model.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test, pred_x))
    
    
    
    return pred_x

In [80]:
sizes = [0.1, 0.16, 0.33]
pred_results = []
pred_results = pd.DataFrame(pred_results)
x = 1
for operator in operators:
    #predict_y(operator, 0.2)
    pred_results[f"X{x}"] = predict_x(operator, 0.2)
    pred_results[f"Y{x}"] = predict_y(operator, 0.2)
    print([predict_x(operator, 0.2), predict_y(operator,0.2)])
    x += 1

[array([187.59301477, 234.69717522, 171.33045545, 161.93733509,
       188.63355757, 176.12305381]), array([104.92144644, 117.09531404, 114.87945977,  99.06080027,
       104.25347967, 106.9843781 ])]
[array([111.13740536,  74.73618779,  90.7381659 ,  94.85186672,
       112.94548331, 101.53270205]), array([116.70980058,  99.80838462, 112.08620604, 139.04635092,
       120.94133633, 112.5509169 ])]
[array([174.86980799, 163.95210787, 115.34383207, 154.35424158,
       154.58993142, 140.29055952]), array([106.48053473,  43.40449935,  76.53041334, 108.00391533,
        99.36894506, 107.01504435])]
[array([222.59324627, 245.92548327, 227.48254486, 174.81500478,
       205.70769187, 205.26848601]), array([141.52668719, 105.37319924, 137.88064478, 144.08338973,
       131.44233705, 146.1038101 ])]
[array([176.61916748, 180.93825158, 162.44669541, 175.30811736,
       178.52299845, 174.0612494 ]), array([109.471628  , 103.11162156, 106.13869805, 107.76610202,
       116.76694859, 114.3735833

In [81]:
pred_results

,X1,Y1,X2,Y2,X3,Y3,X4,Y4,X5,Y5
0,187.593015,104.921446,111.137405,116.709801,174.869808,106.480535,222.593246,141.526687,176.619167,109.471628
1,234.697175,117.095314,74.736188,99.808385,163.952108,43.404499,245.925483,105.373199,180.938252,103.111622
2,171.330455,114.879460,90.738166,112.086206,115.343832,76.530413,227.482545,137.880645,162.446695,106.138698
3,161.937335,99.060800,94.851867,139.046351,154.354242,108.003915,174.815005,144.083390,175.308117,107.766102
4,188.633558,104.253480,112.945483,120.941336,154.589931,99.368945,205.707692,131.442337,178.522998,116.766949
5,176.123054,106.984378,101.532702,112.550917,140.290560,107.015044,205.268486,146.103810,174.061249,114.373583


In [ ]:
pred_results.to_excel("C:\\Users\\stadn\\Desktop\\_robot_dane_operator\\predictions_dla_20.xlsx")

In [82]:
sizes = [0.1, 0.16, 0.33]
preds_for_3 = pd.DataFrame()
preds_for_5 = pd.DataFrame()
preds_for_10 = pd.DataFrame()

for size in sizes:
    x = 1
    for operator in operators:
        if size == 0.1:
            preds_for_3[f"X{x}"] = predict_x(operator, size)
            preds_for_3[f"Y{x}"] = predict_y(operator, size)
            x += 1
            
        elif size == 0.16:
            preds_for_5[f"X{x}"] = predict_x(operator, size)
            preds_for_5[f"Y{x}"] = predict_y(operator, size)
            x += 1
            
        elif size == 0.33:
            preds_for_10[f"X{x}"] = predict_x(operator, size)
            preds_for_10[f"Y{x}"] = predict_y(operator, size)
            x += 1
            
        print([predict_x(operator, size), predict_y(operator, size)])
    



[array([184.04216373, 225.19775862, 170.62075005]), array([106.94941909, 114.4451157 , 114.66214838])]
[array([104.66046958,  75.68226506,  96.22782609]), array([113.33300382,  99.69443346, 111.65606117])]
[array([174.07226192, 166.36354217, 117.0227528 ]), array([106.48578224,  40.8445415 ,  75.80969931])]
[array([220.21907079, -82.29311869, 225.81578813]), array([146.05824184, 102.77146587, 141.58068556])]
[array([175.2749922 , 180.76068119, 162.98094162]), array([109.24766809, 103.27018362, 106.10442501])]
[array([187.58148946, 234.66296538, 171.3322856 , 161.92295427,
       188.6182828 ]), array([105.66650501, 115.13037702, 114.88830362,  99.7818884 ,
       105.52210532])]
[array([110.49089162,  74.72317242,  91.74710581,  94.31866783,
       111.88440631]), array([118.3915978 , 100.00161332, 112.41367931, 141.59268217,
       120.19049918])]
[array([174.88775445, 163.93619194, 115.32512526, 154.38077337,
       154.54444559]), array([107.17475634,  46.23260676,  76.16570234, 108

In [ ]:
pred_results.to_excel("C:\\Users\\stadn\\Desktop\\_robot_dane_operator\\predictions.xlsx")

In [89]:
preds_for_5

,X1,Y1,X2,Y2,X3,Y3,X4,Y4,X5,Y5
0,187.581489,105.666505,110.490892,118.391598,174.887754,107.174756,222.698973,143.070830,176.033941,109.527577
1,234.662965,115.130377,74.723172,100.001613,163.936192,46.232607,190.613937,104.271355,180.648651,103.030614
2,171.332286,114.888304,91.747106,112.413679,115.325125,76.165702,226.886338,139.785207,162.820469,106.091619
3,161.922954,99.781888,94.318668,141.592682,154.380773,108.728221,175.239960,142.834536,174.026883,107.850713
4,188.618283,105.522105,111.884406,120.190499,154.544446,99.031535,205.526553,131.774563,177.414670,117.177976


In [86]:
def predict_y(data, regressor):
    X = data.drop("Y", axis=1)
    y = data["Y"]
    
    if regressor == "polynomial":
        poly_converter = PolynomialFeatures(degree=3, include_bias=False)
        poly_converter.fit(X)
        poly_features = poly_converter.transform(X)

        X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.16, random_state=45)
    
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    
        model = LinearRegression()
        model.fit(X_train, y_train)
        pred_y = model.predict(X_test)
    
        rmse = np.sqrt(mean_squared_error(y_test, pred_y))
        
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.16, random_state=45)
        
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    
    
    if regressor == "linear":
        model = LinearRegression()
        model.fit(X_train, y_train)
        pred_y = model.predict(X_test)
        
        rmse = np.sqrt(mean_squared_error(y_test, pred_y))
        
    elif regressor == "svr":
        model = SVR()
        model.fit(X_train, y_train)
        pred_y = model.predict(X_test)
        
        rmse = np.sqrt(mean_squared_error(y_test, pred_y))
        
    return rmse

In [87]:
def predict_x(data, regressor):
    X = data.drop("X", axis=1)
    y = data["X"]
    
    if regressor == "polynomial":
        poly_converter = PolynomialFeatures(degree=3, include_bias=False)
        poly_converter.fit(X)
        poly_features = poly_converter.transform(X)

        X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.16, random_state=45)
    
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    
        model = LinearRegression()
        model.fit(X_train, y_train)
        pred_x = model.predict(X_test)
    
        rmse = np.sqrt(mean_squared_error(y_test, pred_x))
        
    else:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.16, random_state=45)
        
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    
    
    if regressor == "linear":
        model = LinearRegression()
        model.fit(X_train, y_train)
        pred_x = model.predict(X_test)
        
        rmse = np.sqrt(mean_squared_error(y_test, pred_x))
        
    elif regressor == "svr":
        model = SVR()
        model.fit(X_train, y_train)
        pred_x = model.predict(X_test)
        
        rmse = np.sqrt(mean_squared_error(y_test, pred_x))
        
    return rmse

In [88]:
regressors = ["polynomial", "linear", "svr"]
x = 1
for regressor in regressors:
    for operator in operators:
        print([predict_x(operator, regressor), predict_y(operator, regressor)])

[22.625508442315095, 16.19756856128146]
[27.82195457616771, 24.168304207164233]
[26.669335686718775, 14.536269613156291]
[31.078660333219073, 47.29077603462833]
[8.18817134758506, 5.672174486645051]
[12.917589084665904, 15.103056061766885]
[32.05650041888402, 19.02386185843107]
[19.14818415933013, 13.895377579054376]
[35.20899156656464, 50.744971741364594]
[4.317485477995652, 5.813597673434679]
[16.930962722439325, 14.576887491727799]
[31.88102396035853, 23.121191986480994]
[16.584172797640964, 22.49264251344584]
[17.376424994537278, 57.042690430054684]
[4.109864188160266, 6.583060199657863]
